In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np

# proFSA
import fsa.load as load
import fsa.spatial as spatial
import fsa.sequence as seq
import fsa.elastic as elastic

In [2]:
# Selected values from the GUI

spatial_alignment = True
WEIGHT_METHOD = "Binary"
SELECTION = "Intersection"

# Binary: params = radius
if WEIGHT_METHOD == "Binary":
    params = [10.]
    
    if SELECTION == "Relaxed":
        method = "intersect"
        structure_list = ["rel"]
        
    elif SELECTION == "Deformed":
        method = "intersect"
        structure_list = ["def"]
    
    elif SELECTION == "Union":
        method = "union"
        structure_list = ["rel", "def"]
    
    elif SELECTION == "Intersection":
        method = "intersect"
        structure_list = ["rel", "def"]
    
# Linear: params = inner radius, outter radius
elif WEIGHT_METHOD == "Linear":
    params = [10., 11.]
    
    if SELECTION == "Relaxed":
        method = "linear"
        structure_list = ["rel"]
        
    elif SELECTION == "Deformed":
        method = "linear"
        structure_list = ["def"]
    
    elif SELECTION == "Average":
        method = "linear"
        structure_list = ["rel", "def"]    

# N nearest: params = neighbouts
elif WEIGHT_METHOD == "N nearest":
    params = [3]
    
    if SELECTION == "Relaxed":
        method = "minimal"
        structure_list = ["rel"]
        
    elif SELECTION == "Deformed":
        method = "minimal"
        structure_list = ["def"]


In [3]:
# Input
rel_pdb, def_pdb = '6FKI', '6FKF'

# Load
rel_pps = load.single_structure(rel_pdb)
def_pps = load.single_structure(def_pdb)

In [4]:
# Sequence alignment
com_res, rel_dict, def_dict = seq.pairwise_alignment(rel_pps, def_pps)

In [5]:
# Spatial alignment
if spatial_alignment:
    spatial.align_structures(rel_pps,
                             def_pps,
                             common_res = com_res,
                             rel_dict = rel_dict,
                             def_dict = def_dict)

In [6]:
# Coordinates
rel_xyz, rel_labels = load.coordinates(rel_pps,
                                       com_res,
                                       rel_dict)

def_xyz, def_labels = load.coordinates(def_pps,
                                       com_res,
                                       def_dict)


In [7]:
# Strain prepare
xyz_list = []
if "rel" in structure_list:
    xyz_list.append(rel_xyz)
if "def" in structure_list:
    xyz_list.append(def_xyz)

weights = elastic.compute_weights_fast(xyz_list,
                                       method = method,
                                       parameters = params)

F = elastic.deformation_gradient_fast(weights, rel_xyz, def_xyz)

In [8]:
# Strain results
rot_angle, rot_axis = elastic.rotations(F)
rot_axis = spatial.vector_to_polar(rel_xyz, rot_axis)
_, gam_n = elastic.lagrange_strain(F)
stretches, stretch_axis = elastic.principal_stretches_from_g(gam_n)

In [9]:
# Save to PDB
strain_cluster_dict = {}
for label, st in zip(rel_labels, np.abs(stretches[:, 0])):
    strain_cluster_dict[label[0]] = st

load.save_bfactor(name=rel_pdb,
                  bfact=strain_cluster_dict,
                  ext='_st_quant_L0')

In [10]:
# Create dictionaries and files with principal stretches and rotation angles
var_values = [np.abs(stretches[:, 0]), 
              np.abs(stretches[:, 1]), 
              np.abs(stretches[:, 2]), 
              rot_angle]

load.save_txt(rel_pdb, def_pdb, var_values, rel_labels, "l1-l2-l3-rot")